In [2]:
import torch
import torch.nn as nn
import torch.optim as optim

def make_batch():
    input_batch = []
    target_batch = []
    
    for sen in sentences:
        word = sen.split() #공백을 기준으로 토큰화
        input = [word_dict[n] for n in word[:-1]] #제일 끝 요소 빼고 input으로 넣음
        target = word_dict[word[-1]] #causal language modeling으로 제일 마지막 단어를 예측하도록 하는 모델
        
        input_batch.append(input)
        target_batch.append(target)
        
    return input_batch, target_batch

#Model
class NNLM(nn.Module):
    def __init__(self):
        super(NNLM, self).__init__() #super():상위 클래스 상속
        self.C = nn.Embedding(n_class, m)
        self.H = nn.Linear(n_step * m, n_hidden, bias=False) #bias=False를 사용하면 parameter의 수가 감소하여 모델 단순화 가능
        self.d = nn.Parameter(torch.ones(n_hidden)) #nn.Parameter : nn.Module 안에 만들어진 tensor들을 보관가능
        self.U = nn.Linear(n_hidden, n_class, bias=False)
        self.W = nn.Linear(n_step*m, n_class, bias=False)
        self.b = nn.Parameter(torch.ones(n_class))
        
    def forward(self,X):
        X = self.C(X) #X = [batch_size, n_step, m]
        X = X.view(-1, n_step*m)
        tanh = torch.tanh(self.d + self.H(X))
        output = self.b + self.W(X) + self.U(tanh)
        return output
    
if __name__ == '__main__': #main함수의 선언, 시작을 의미한다
    n_step = 2 #step의 수, n-1 in paper
    n_hidden = 2
    m = 2 #embedding size
    
    sentences = ['i like dog', 'i love coffee', 'i hate milk']
    
    word_list = ' '.join(sentences).split()
    word_list = list(set(word_list))
    word_dict = {w: i for i,w in enumerate(word_list)}
    number_dict = {i:w for i, w in enumerate(word_list)}
    n_class = len(word_dict) #vocabulary의 단어 수
    
    model = NNLM()
    
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=.001)
    
    input_batch, target_batch = make_batch()
    input_batch = torch.LongTensor(input_batch)
    target_batch = torch.LongTensor(target_batch)
    
    #학습 시작
    for epoch in range(5000):
        optimizer.zero_grad()
        output = model(input_batch)
        
        #output : [batch_size, n_class], target_batch : [batch_size]
        loss = criterion(output, target_batch)
        if (epoch +1) % 1000 == 0:
            print('Epoch:', '%04d' % (epoch +1), 'cost =', '{:.6f}'.format(loss))
        
        loss.backward()
        optimizer.step()
        
    #Prediction
    predict = model(input_batch).data.max(1, keepdim=True)[1]
    
    #Test
    print([sen.split()[:2] for sen in sentences], '->', [number_dict[n.item()] for n in predict.squeeze()])
        

Epoch: 1000 cost = 0.095607
Epoch: 2000 cost = 0.016130
Epoch: 3000 cost = 0.004702
Epoch: 4000 cost = 0.001830
Epoch: 5000 cost = 0.000830
[['i', 'like'], ['i', 'love'], ['i', 'hate']] -> ['dog', 'coffee', 'milk']
